# Mapping Tables

In this notebook, we will create a mapping table which provides us with additional information for analysis without affecting our transactional processing system.

A true mapping table would be a completely separate table, but in this case, we opted to add a nullable lookup ID on the `IncidentCode` table because we knew it would not be disruptive to the current process.

## Create the New Column

The first thing we need to do is to create a new column on `IncidentCode` called `IncidentTypeID`.  It will correspond to an `IncidentType` table that we will soon create.

In [1]:
IF NOT EXISTS
(
    SELECT 1
    FROM sys.columns sc
    WHERE
        sc.name = N'IncidentTypeID'
        AND sc.object_id = OBJECT_ID('dbo.IncidentCode')
)
BEGIN
    ALTER TABLE dbo.IncidentCode ADD IncidentTypeID INT NULL;
END
GO

## Create the New Table

After creating the `IncidentTypeID` column, we will want to link it up to a new table.  The `IncidentType` table will let us group together different incident description codes into a smaller group so we can perform analysis more easily.  As we can see from a quick `TOP 15 *` of the `IncidentCode` table, the specific police codes fit into relatively easy to define categories such as forgery, murder, and fraud.

<img src="https://raw.githubusercontent.com/feaselkl/TidyData/master/Images/IncidentType.png" width="500" />

With this information, we can create a new table for incident types.

In [2]:
IF NOT EXISTS
(
    SELECT 1
    FROM sys.tables t
    WHERE
        t.name = N'IncidentType'
)
BEGIN
    CREATE TABLE dbo.IncidentType
    (
        IncidentTypeID INT IDENTITY(1,1) NOT NULL,
        IncidentType VARCHAR(55) NOT NULL
    );
    ALTER TABLE dbo.IncidentType ADD CONSTRAINT [PK_IncidentType] PRIMARY KEY CLUSTERED(IncidentTypeID);
    ALTER TABLE dbo.IncidentType ADD CONSTRAINT [UKC_IncidentType] UNIQUE(IncidentType);
    ALTER TABLE dbo.IncidentCode ADD CONSTRAINT [FK_IncidentCode_IncidentType]
        FOREIGN KEY(IncidentTypeID)
        REFERENCES dbo.IncidentType(IncidentTypeID);
END
GO

## Notes on Building a Mapping Table

Now comes the hard part:  creating incident types and linking incident descriptions to those types.  Having surveyed the data (and having no outside guidance on what those types **ought** to be), we can build out incident types from the incident descriptions by following a set of mapping rules.

### Separator Characters

There are three key separator chracters:  `/`, `-`, and `(`.  If we see one of these characters in a description, we know that the incident type is the text **before** these characters.

### Special Cases

There are a few special cases in this data set.

1. [something] W/[something] should just be [something].  For example, `ROBBERY W/FLOUNDER` should just be `ROBBERY`.
2. If the block starts with `MURDER`, `EMBEZZLEMENT`, `ASSAULT`, `LARCENY`, or `RAPE`, that is the respective incident type.  Each of these cases has examples which do not have a separator character, so we will need to take care of them specially.

### Cleanup

After we figure out the incident type, we will want to trim any whitespace around the incident type, as "MURDER" and "MURDER &nbsp;&nbsp;" are the same thing.

We will also want to get the distinct values so that we have one `MURDER` instead of ten.

### Tips and Tricks

This script makes heavy use of the `APPLY` operator.  If you want to see the `APPLY` operator in action, I have an entire talk on it at https://csmore.info/on/apply.  Here we will use the `APPLY` operator to work methodically, making it easier for us to visualize changes and come up with a viable solution.  You do not strictly need to use it, but I think the answer without `APPLY` will be considerably harder to understand and probably will perform no better.

We also use `CHARINDEX` and `PATINDEX` to find the positions of characters.  If you are not familiar with them, they can be quite useful functions.

## Step One:  Create a Temp Table



In [4]:
DROP TABLE IF EXISTS #IncidentType;
CREATE TABLE #IncidentType
(
	IncidentCode VARCHAR(5),
	IncidentDescription VARCHAR(55),
	IncidentType VARCHAR(55),
	IncidentTypeID INT
);

## Step Two:  Load Incident Type

This script loads one record per incident code in the `IncidentCode` table.  For each incident code, we run through the rules above to determine what the incident type ought to be.

In [5]:
INSERT INTO #IncidentType
(
	IncidentCode,
	IncidentDescription,
	IncidentType
)
SELECT
	ic.IncidentCode,
	ic.IncidentDescription,
	LTRIM(RTRIM(SUBSTRING(ic.IncidentDescription, 1, fdc.FirstDecidingCharacter - 1))) AS IncidentType
FROM dbo.IncidentCode ic
	CROSS APPLY(SELECT REPLACE(ic.IncidentDescription, ' W/', '-W/') AS IncidentDescription) idW
	CROSS APPLY(SELECT REPLACE(idW.IncidentDescription, 'MURDER ', 'MURDER/') AS IncidentDescription) idM
	CROSS APPLY(SELECT REPLACE(idM.IncidentDescription, 'EMBEZZLEMENT ', 'EMBEZZLEMENT/') AS IncidentDescription) idE
	CROSS APPLY(SELECT REPLACE(idE.IncidentDescription, 'ASSAULT ', 'ASSAULT/') AS IncidentDescription) idA
	CROSS APPLY(SELECT REPLACE(idA.IncidentDescription, 'LARCENY ', 'LARCENY/') AS IncidentDescription) idL
	CROSS APPLY(SELECT REPLACE(idL.IncidentDescription, 'RAPE ', 'RAPE/') AS IncidentDescription) idFinal
	CROSS APPLY(SELECT CHARINDEX('/', idFinal.IncidentDescription, 1) AS FirstSlash) fs
	CROSS APPLY(SELECT CHARINDEX('-', idFinal.IncidentDescription, 1) AS FirstHyphen) fh
	CROSS APPLY(SELECT CHARINDEX('(', idFinal.IncidentDescription, 1) AS FirstParens) fp
	CROSS APPLY
	(
		SELECT
			CASE WHEN fs.FirstSlash = 0 THEN 999 ELSE fs.FirstSlash END AS FirstSlash,
			CASE WHEN fh.FirstHyphen = 0 THEN 999 ELSE fh.FirstHyphen END AS FirstHyphen,
			CASE WHEN fp.FirstParens = 0 THEN 999 ELSE fp.FirstParens END AS FirstParens
	) f
	CROSS APPLY
	(
		SELECT
			CASE
				WHEN f.FirstSlash < f.FirstHyphen AND f.FirstSlash < f.FirstParens THEN f.FirstSlash
				WHEN f.FirstParens < f.FirstHyphen AND f.FirstParens < f.FirstSlash THEN f.FirstParens
				ELSE f.FirstHyphen
			END AS FirstDecidingCharacter
	) fdc;

At this point, we have all of the incident types and can preview them to ensure that things look like how we would expect.

In [6]:
SELECT TOP(15)
    it.IncidentCode,
    it.IncidentDescription,
    it.IncidentType
FROM #IncidentType it;

## Build Incident Type IDs

Now we can build up a set of incident type IDs.  This code is a bit complicated so let's walk through it step by step.

### Finding Existing Types

It's possible that we've run this code before and have a set of incident type IDs.  In order to make sure we don't accidentally assign new values to those IDs, we will get the current values from the `IncidentType` table in a Common Table Expression (CTE) called `existingTypes`.  We will also get the maximum value for `IncidentTypeID` to ensure that we don't clobber an existing ID value.

### Creating Incident Type IDs

The `itID` CTE looks for cases where we do **not** have an existing incident type in our table and assigns new IDs in alphabetical order.  We use the `DENSE_RANK` window function to ensure contiguous numbering for incident types regardless of how many rows there are for a particular value.

### Creating the New ID Set

The `newID` CTE combines together our existing incident types along with the newly-created incident types as one bundle.

### Updating Incident Type IDs

Once we have the `newID` CTE put together, we can join back to the `#IncidentType` temp table and set the `IncidentTypeID` value.

In [7]:
DECLARE
    @MaxIncidentTypeID TINYINT = 0;

SELECT
    @MaxIncidentTypeID = MAX(it.IncidentTypeID)
FROM dbo.IncidentType it;

WITH existingTypes AS
(
    SELECT
        it.IncidentTypeID,
        it.IncidentType
    FROM dbo.IncidentType it
),
itID AS
(
	SELECT
		i.IncidentType,
		DENSE_RANK() OVER (ORDER BY i.IncidentType)  AS IncidentTypeID
	FROM #IncidentType i
        LEFT OUTER JOIN existingTypes et
            ON i.IncidentType = et.IncidentType
    WHERE
        et.IncidentType IS NULL
),
newIDs AS
(
    SELECT DISTINCT
        itID.IncidentType,
        itID.IncidentTypeID + @MaxIncidentTypeID AS IncidentTypeID
    FROM itID

    UNION ALL

    SELECT
        et.IncidentType,
        et.IncidentTypeID
    FROM existingTypes et
)
UPDATE it
SET
	IncidentTypeID = i.IncidentTypeID
FROM #IncidentType it
	INNER JOIN newIDs i
		ON it.IncidentType = i.IncidentType;


## Insert New Incident Type IDs

Now that we have our incident types, we can insert the new values into the `IncidentType` table.  For values which already exist, we won't need to do anything.


In [8]:
SET IDENTITY_INSERT dbo.IncidentType ON;
INSERT INTO dbo.IncidentType
(
	IncidentTypeID,
	IncidentType
)
SELECT DISTINCT
	it.IncidentTypeID,
	it.IncidentType
FROM #IncidentType it
    LEFT OUTER JOIN dbo.IncidentType i
        ON it.IncidentTypeID = i.IncidentTypeID
WHERE
    i.IncidentTypeID IS NULL;
SET IDENTITY_INSERT dbo.IncidentType OFF;

## Update Incident Codes

All of this work serves the purpose of setting incident types on the `IncidentCode` table.  Now that we have all of the groundwork in place, we can join back to the `#IncidentType` temp table and set each code's `IncidentTypeID`.

In [9]:
UPDATE ic
SET
	IncidentTypeID = it.IncidentTypeID
FROM dbo.IncidentCode ic
	INNER JOIN #IncidentType it
		ON ic.IncidentCode = it.IncidentCode;
GO

In [10]:
SELECT TOP(15)
    ic.IncidentCode,
    ic.IncidentDescription,
    ic.IncidentTypeID
FROM dbo.IncidentCode ic;